In [1]:
import params

import os
import wandb
from tqdm import tqdm

In [2]:
wandb.login()

wandb: Currently logged in as: gianfranco-romani. Use `wandb login --relogin` to force relogin


True

In [5]:
path = '/home/gianfree/Desktop/CISI-project-MLOps/'
dataset_texts_path = "./CISI.ALL"
dataset_queries_path = "./CISI.QRY"
dataset_mapping_path = "./CISI.REL"

In [6]:
#Read data from CISI.ALL file and store in dictionary 
with open(os.path.join(path, 'CISI.ALL')) as f:
        lines = ""
        for l in f.readlines():
            lines += "\n" + l.strip() if l.startswith(".") else " " + l.strip()
        lines = lines.lstrip("\n").split("\n")
 
doc_set = {}
doc_id = ""
doc_text = []

for l in lines:
    if l.startswith(".I"):
        doc_id = l.split(" ")[1].strip() 
    elif l.startswith(".X"):
        doc_set[doc_id] = doc_text#.lstrip(" ")
        doc_id = ""
        doc_text = []
        continue
    elif l.startswith(".T"):
        doc_text.append(l.strip()[3:])
    elif l.startswith(".W"):
        doc_text.append(l.strip()[3:])

print(f"Number of documents = {len(doc_set)}\n")
print(doc_set["1"]) 
print("#"*100)

#Read data from CISI.QRY file and store in dictionary 
with open(os.path.join(path, 'CISI.QRY')) as f:
        lines = ""
        for l in f.readlines():
            lines += "\n" + l.strip() if l.startswith(".") else " " + l.strip()
        lines = lines.lstrip("\n").split("\n")
          
qry_set = {}
qry_id = ""
for l in lines:
    if l.startswith(".I"):
        qry_id = l.split(" ")[1].strip() 
    elif l.startswith(".W"):
        qry_set[qry_id] = l.strip()[3:]
        qry_id = ""

print(f"\nNumber of queries = {len(qry_set)}\n")    
print(qry_set["1"]) 
print("#"*100)

#Read data from CISI.REL file and store in dictionary 
rel_set = {}
with open(os.path.join(path, 'CISI.REL')) as f:
    for l in f.readlines():
        qry_id = l.lstrip(" ").strip("\n").split("\t")[0].split(" ")[0] 
        doc_id = l.lstrip(" ").strip("\n").split("\t")[0].split(" ")[-1]

        if qry_id in rel_set:
            rel_set[qry_id].append(doc_id)
        else:
            rel_set[qry_id] = []
            rel_set[qry_id].append(doc_id)

print(f"\nNumber of mappings = {len(rel_set)}")
print(rel_set["1"]) 

Number of documents = 1460

['18 Editions of the Dewey Decimal Classifications', "The present study is a history of the DEWEY Decimal Classification.  The first edition of the DDC was published in 1876, the eighteenth edition in 1971, and future editions will continue to appear as needed.  In spite of the DDC's long and healthy life, however, its full story has never been told.  There have been biographies of Dewey that briefly describe his system, but this is the first attempt to provide a detailed history of the work that more than any other has spurred the growth of librarianship in this country and abroad."]
####################################################################################################

Number of queries = 112

What problems and concerns are there in making up descriptive titles? What difficulties are involved in automatically retrieving articles from approximate titles? What is the usual relevance of the content of articles to their titles?
##################

In [7]:
run = wandb.init(project=params.WANDB_PROJECT, entity=params.ENTITY, job_type="upload")
artifact = wandb.Artifact(name="cisi_project", type="dataset")

In [8]:
artifact.add_file(local_path='./CISI.ALL', name='questions')
artifact.add_file(local_path='./CISI.QRY', name='queries')
artifact.add_file(local_path='./CISI.REL', name='mappings')

<ManifestEntry digest: 9X77Pr4PBhAtaf4J9116cA==>

In [11]:
table_questions = wandb.Table(columns=["ID", "Title", "Text"])
for i, doc in tqdm(enumerate(doc_set.items()), total=len(doc_set)):
    table_questions.add_data(doc[0], doc[1][0], doc[1][1])

artifact.add(table_questions, "table_questions")

100%|████████████████████████████████████| 1460/1460 [00:00<00:00, 14458.51it/s]


<ManifestEntry digest: U+B+IFh6j8qGtGhkGDZmNA==>

In [12]:
run.log_artifact(artifact)
run.finish()

In [13]:
artifact = run.use_artifact('cisi_project')

# Download the artifact's contents
artifact_dir = artifact.download()

CommError: Attempted to fetch artifact without alias (e.g. "<artifact_name>:v3" or "<artifact_name>:latest")